<a href="https://colab.research.google.com/github/dzeke/ColoradoRiverCollaborate/blob/main/Powell10year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lake Powell Evaporation
## David E. Rosenberg
## February 13, 2024

## Overview
This is a Notebook (.ipynb). This document reads daily Reclamation data for Lake Powell Evaporation (https://www.usbr.gov/uc/water/hydrodata/reservoir_data/919/csv/25.csv). The code aggregates data to yearly (water year starting Oct 1). We then plot:

 + Evaporation by year over period of record
 + Total evaporation for each period of consecutative 10 years

## Requested Citation
David E. Rosenberg (2021), “Lake Powell Evaporation.” Utah State University. Logan, Utah.
https://github.com/dzeke/ColoradoRiverCollaborate/tree/main/Powell10year/AsPython.

### Pandas Cheat Sheet
Pandas cheat sheet at https://pandas.pydata.org/docs/user_guide/10min.html.

In [6]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

# Add this command on first run of cell
#!pip install plotnine
from plotnine import *
from plotnine.data import mpg
%matplotlib inline

# Load the data from a csv file on Github
#EvapDataAsUrl = 'https://raw.githubusercontent.com/dzeke/ColoradoRiverCollaborate/main/Powell10year/AsPython/25.csv'

# Load the data from Reclamation's webservice
EvapDataAsUrl = 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/919/csv/25.csv'
# Move the Dataset into a Pandas Dataframe
dfEvap = pd.read_csv(EvapDataAsUrl)
# Read the number of rows
nRows = len(dfEvap)

# Convert the datetime object column to pandas date time
dfEvap['date'] = pd.to_datetime(dfEvap['datetime'])

#Add year and month columns
dfEvap['year'] = pd.DatetimeIndex(dfEvap['date']).year
dfEvap['month'] = pd.DatetimeIndex(dfEvap['date']).month

#Calculate the water year, Increment by 1 year, Months 10, 11, 12
dfEvap['wateryear'] = dfEvap['year']
dfEvap.loc[dfEvap['month'] >= 10, 'wateryear'] = dfEvap['year'] + 1

#Calculate number of days/records for each water year
dfDaysPerWaterYear = pd.DataFrame(dfEvap.groupby('wateryear').size().reset_index())
# Rename the 0 column to DaysPerYear
dfDaysPerWaterYear.columns = ['wateryear', 'DaysPerYear']
#Filter to years with more than 360 days / records
dfFullYears = dfDaysPerWaterYear.query('DaysPerYear >= 360')

#print(dfFullYears.head())
print(dfEvap.dtypes)
print(dfEvap.head())
#Calculate total evaporation for each water year
dfEvapWaterYear = dfEvap.groupby('wateryear').sum().reset_index()

# Left join to only include Water Years with more than 360 days
dfEvapWaterYear = dfFullYears.merge(dfEvapWaterYear, on='wateryear', how='left')

# Turn acre-feet into million acre-feet
dfEvapWaterYear.evaporation = dfEvapWaterYear.evaporation / 1e6
# convert back to a dataframe
dfEvapWaterYear = pd.DataFrame(dfEvapWaterYear)

print()
print(dfEvapWaterYear.dtypes)
print(dfEvapWaterYear.head())

#dfEvapWaterYear['date'] = pd.to_datetime(dfEvapWaterYear['wateryear'])



datetime               object
evaporation           float64
date           datetime64[ns]
year                    int32
month                   int32
wateryear               int32
dtype: object
     datetime  evaporation       date  year  month  wateryear
0  1963-03-14          0.0 1963-03-14  1963      3       1963
1  1963-03-15          0.0 1963-03-15  1963      3       1963
2  1963-03-16          0.0 1963-03-16  1963      3       1963
3  1963-03-17          0.0 1963-03-17  1963      3       1963
4  1963-03-18          0.0 1963-03-18  1963      3       1963


TypeError: datetime64 type does not support sum operations

# Figure 1. Annual Lake Powell Evaporation

In [ ]:
# Use matplotlib

import matplotlib.pyplot as plt

plt.bar(dfEvapWaterYear.wateryear, dfEvapWaterYear.evaporation, color = "grey", width = 1)
plt.xlabel('')
plt.ylabel('Evaporation\n(million acre-feet)')
plt.grid(axis = 'y', linestyle = "dotted")

#Use ggplot
###(ggplot(dfEvapWaterYear, aes(x = 'wateryear'))
### + geom_bar(aes(y='evaporation'), color = "grey", fill = "grey", stat="identity")
 # geom_line(aes(y=OneYearRelease, group = 1, color="Release"), size=2) +
#  + scale_color_manual("", values = "grey50")
#  + scale_fill_manual("", values = "grey50")
### + labs(x="", y="Evaporation\n(million acre-feet per year)")

 # + theme(text = element_text(size=20), legend.title=element_blank(), legend.text=element_text(size=18),
#        legend.key = element_blank())
###)


# Figure 2. Evaporation each consecutive 10-years

In [ ]:
# Calculate a rolling 10-year sum
dfEvap10year = dfEvapWaterYear
dfEvap10year["Evap10yr"] = dfEvap10year['evaporation'].rolling(10).sum()

print(dfEvap10year.head())

plt.bar(dfEvap10year.wateryear, dfEvap10year.Evap10yr, color = "grey", width = 1)
plt.xlabel('')
plt.ylabel('10-Year Evaporation\n(million acre-feet)')
plt.grid(axis = 'y', linestyle = "dotted")